In [1]:
from data_reader import Vocabulary, HWDBDatasetHelper, LMDBReader

# your path to data
train_path = r'/DATA/ichuviliaeva/ocr_data/train.lmdb'
test_path = r'/DATA/ichuviliaeva/ocr_data/test.lmdb'
gt_path = r'/DATA/ichuviliaeva/ocr_data/gt.txt'

In [2]:
import cv2
import numpy as np
from centerloss import CenterLoss

/home/ichuviliaeva/miniconda3/envs/ocr_course/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
train_reader = LMDBReader(train_path)
train_reader.open()
train_helper = HWDBDatasetHelper(train_reader)

In [4]:
train_helper, val_helper = train_helper.train_val_split()

In [5]:
train_helper.size(), val_helper.size()

(2578433, 644609)

In [6]:
class pad_prune(object):
    
    def __init__(self, output_w, output_h):
        self.output_w = output_w
        self.output_h = output_h

    def __call__(self, image):
        
        w, h = image.shape
        diff_w = w - self.output_w
        diff_h = h - self.output_h
        masked = np.zeros((self.output_w, self.output_h))
        if diff_w <= 0:
            if diff_h <= 0:
                masked[:w, :h] = image
            else:
                masked[:w, :] = image[:, (diff_h // 2):(h - (diff_h - diff_h // 2))]
            
        if diff_w > 0:
            if diff_h > 0:
                masked = image[(diff_w // 2):(w - (diff_w - diff_w // 2)), 
                               (diff_h // 2):(h - (diff_h - diff_h // 2))]
            else:
                masked[:, :h] = image[(diff_w // 2):(w - (diff_w - diff_w // 2)), :]
        return masked - 127.5 / 255.0  

In [7]:
import torch

from torch.utils.data import Dataset, DataLoader
from torch import nn

class HWDBDataset(Dataset):
    def __init__(self, helper: HWDBDatasetHelper, transform = None):
        self.helper = helper
        self.transform = transform
    
    def __len__(self):
        return self.helper.size()
    
    def __getitem__(self, idx):
        img, label = self.helper.get_item(idx)
        if self.transform:
            img = self.transform(img)
        return img, label

In [8]:
to_one_shape = pad_prune(100, 100)
train_dataset = HWDBDataset(train_helper, to_one_shape)
val_dataset = HWDBDataset(val_helper, to_one_shape)

In [9]:
class SimpleBlock(nn.Module):
        
    def __init__(self, in_channels, out_channels, kernel_size, padding, stride, is_projection = False):
        super().__init__()
        self.bn1 = nn.BatchNorm2d(num_features = in_channels)
        self.relu1 = nn.ReLU()
        self.conv1 = nn.Conv2d(in_channels = in_channels, out_channels = out_channels, 
                               kernel_size = kernel_size, padding = padding, stride = stride)
            
        self.bn2 = nn.BatchNorm2d(num_features = out_channels)
        self.relu2 = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels = out_channels, out_channels = out_channels,
                               kernel_size = kernel_size, padding = padding, stride = stride)
        if is_projection:
            self.project = nn.Conv2d(in_channels = in_channels, out_channels = out_channels,
                               kernel_size = 1, padding = 0, stride = stride)
        else:
            self.project = lambda x: x
            
    def forward(self, x):            
        out = self.conv1(self.relu1(self.bn1(x)))
        out = self.conv2(self.relu2(self.bn2(out)))
        return out + self.project(x)

In [10]:
class TheNet(nn.Module):
    def __init__(self, n_classes):
        super(TheNet, self).__init__()
        self.nn = nn.Sequential(
            nn.Conv2d(in_channels = 1, out_channels = 4, kernel_size = 5, stride = 2),
            nn.ReLU(),
            
            SimpleBlock(in_channels = 4, out_channels = 4, kernel_size = 3, padding = 1, stride = 1),
            SimpleBlock(in_channels = 4, out_channels = 4, kernel_size = 3, padding = 1, stride = 1, 
                             is_projection = True),
            
            nn.Conv2d(4, 8, kernel_size=3, padding=1),
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.Conv2d(8, 8, kernel_size=3, padding=1),
            nn.MaxPool2d((2, 2)),
           
            SimpleBlock(in_channels = 8, out_channels = 8, kernel_size = 3, padding = 1, stride = 1),
            SimpleBlock(in_channels = 8, out_channels = 8, kernel_size = 3, padding = 1, stride = 1, 
                             is_projection = True),
            
            nn.Conv2d(8, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Conv2d(16, 16, kernel_size=3, padding=1),
            nn.MaxPool2d((2, 2)),
            
            SimpleBlock(in_channels = 16, out_channels = 16, kernel_size = 3, padding = 1, stride = 1),
            SimpleBlock(in_channels = 16, out_channels = 16, kernel_size = 3, padding = 1, stride = 1, 
                             is_projection = True),
            
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.MaxPool2d((2, 2)),
            
            SimpleBlock(in_channels = 32, out_channels = 32, kernel_size = 3, padding = 1, stride = 1),
            SimpleBlock(in_channels = 32, out_channels = 32, kernel_size = 3, padding = 1, stride = 1, 
                             is_projection = True),
            
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.MaxPool2d((2, 2)),
            
            nn.Flatten(),
            nn.Linear(3 * 3 * 32, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            
        )
        
        self.last_layer = nn.Linear(128, n_classes, bias=False)
            
    def forward(self, x):
        x = self.nn(x)
        y = self.last_layer(x)
        return x, y

In [11]:
model = TheNet(train_helper.vocabulary.num_classes())
model.eval()
res = model(torch.tensor(train_dataset[0][0], dtype=torch.float32).view(1, 1, 100, 100))
print(res[0].shape)
print(res[1].shape)

torch.Size([1, 128])
torch.Size([1, 7330])


In [12]:
model = model.cuda()

In [13]:
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, drop_last=True, num_workers=8)
val_loader = DataLoader(val_dataset, batch_size=2048, shuffle=False, num_workers=8)

In [14]:
optim = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()

In [15]:
center_loss = CenterLoss(num_classes=train_helper.vocabulary.num_classes(), feat_dim=128, use_gpu=True)
optimizer_centloss = torch.optim.Adam(center_loss.parameters(), lr=0.3)

In [16]:
from tqdm import tqdm


def run_validation(val_loader: DataLoader, model: nn.Module, n_steps=None):
    model.eval()
    n_good = 0
    n_all = 0
    wrapper = lambda x: x
    if n_steps is None:
        n_steps = len(val_loader)
        wrapper = tqdm
    
    with torch.no_grad():
        for batch, (X, y) in enumerate(wrapper(val_loader)):
            if batch == n_steps:
                break
            features, logits = model(X.unsqueeze(1).to(torch.float32).cuda())
            classes = torch.argmax(logits, dim=1).cpu().numpy()
            n_good += sum(classes == y.cpu().numpy())
            n_all += len(classes)
    
    return n_good / n_all


def train_epoch(train_loader: DataLoader, val_loader: DataLoader, model: nn.Module, optim, loss_fn, alpha = 1.0):
    for batch, (X, y) in enumerate(tqdm(train_loader)):
        model.train()
        features, logits = model(X.unsqueeze(1).to(torch.float32).cuda())
        labels = y.to(torch.long).cuda()
        loss = center_loss(features, labels) * alpha + loss_fn(logits, labels)
        
        optim.zero_grad()
        optimizer_centloss.zero_grad()
        
        loss.backward()
        optim.step()
        
        for param in center_loss.parameters():
             param.grad.data *= (1./alpha)

        optimizer_centloss.step()

In [25]:
for epoch in range(11):
    print(f'Epoch {epoch}:')
    train_epoch(train_loader, val_loader, model, optim, loss_fn, alpha = 0.3)
    accuracy = run_validation(val_loader, model)
    print(f'accuracy: {accuracy}')
    if epoch % 5 == 0:
        torch.save(model.state_dict(), f'arch1__center_epoch{epoch}.pth')

Epoch 0:


100%|█████████████████████████████████████████████████████████████████████| 315/315 [00:51<00:00,  6.10it/s]


accuracy: 0.033614175414863896
Epoch 1:


100%|█████████████████████████████████████████████████████████████████████| 315/315 [00:52<00:00,  6.04it/s]


accuracy: 0.5337700838803058
Epoch 2:


...

accuracy: 0.8732875277881631
Epoch 9:


100%|█████████████████████████████████████████████████████████████████████| 315/315 [00:49<00:00,  6.39it/s]


accuracy: 0.8812148139414746
Epoch 10:


100%|█████████████████████████████████████████████████████████████████████| 315/315 [00:55<00:00,  5.69it/s]

accuracy: 0.8855880076139179


In [32]:
epoch = 11
while epoch < 21:
    print(f'Epoch {epoch}:')
    train_epoch(train_loader, val_loader, model, optim, loss_fn, alpha = 0.3)
    accuracy = run_validation(val_loader, model)
    print(f'accuracy: {accuracy}')
    epoch += 1
    if epoch % 5 == 0:
        torch.save(model.state_dict(), f'arch1__center_epoch{epoch}.pth')

Epoch 11:


100%|█████████████████████████████████████████████████████████████████████| 315/315 [00:47<00:00,  6.66it/s]


accuracy: 0.8907213520133911
Epoch 12:


100%|█████████████████████████████████████████████████████████████████████| 315/315 [00:55<00:00,  5.68it/s]


accuracy: 0.8938224567140701
Epoch 13:


...

accuracy: 0.9108700002637258
Epoch 19:


100%|█████████████████████████████████████████████████████████████████████| 315/315 [00:55<00:00,  5.63it/s]


accuracy: 0.9056404735273631
Epoch 20:


100%|█████████████████████████████████████████████████████████████████████| 315/315 [00:47<00:00,  6.64it/s]

accuracy: 0.9131256311965859


In [17]:
 model.load_state_dict(torch.load(f'arch1__center_epoch20.pth'))

<All keys matched successfully>

In [18]:
epoch = 21
while epoch < 31:
    print(f'Epoch {epoch}:')
    train_epoch(train_loader, val_loader, model, optim, loss_fn, alpha = 0.3)
    accuracy = run_validation(val_loader, model)
    print(f'accuracy: {accuracy}')
    epoch += 1
    if epoch % 5 == 0:
        torch.save(model.state_dict(), f'arch1__center_epoch{epoch}.pth')

Epoch 21:


100%|█████████████████████████████████████████████████████████████████████| 315/315 [00:51<00:00,  6.07it/s]


accuracy: 0.9123096326610395
Epoch 22:


100%|█████████████████████████████████████████████████████████████████████| 315/315 [00:51<00:00,  6.12it/s]


accuracy: 0.9139090518438309
Epoch 23:


...

accuracy: 0.9204385914562161
Epoch 29:


100%|█████████████████████████████████████████████████████████████████████| 315/315 [00:33<00:00,  9.27it/s]


accuracy: 0.9220736911833375
Epoch 30:


100%|█████████████████████████████████████████████████████████████████████| 315/315 [00:32<00:00,  9.65it/s]

accuracy: 0.9221326416478827


In [19]:
test_path = r'/DATA/ichuviliaeva/ocr_data/test.lmdb'
pred_path = './pred.txt'

test_reader = LMDBReader(test_path)
test_reader.open()
test_helper = HWDBDatasetHelper(test_reader, prefix='Test')

In [20]:
test_dataset = HWDBDataset(test_helper, to_one_shape)
test_loader = DataLoader(test_dataset, batch_size=2048, shuffle=False, num_workers=8)

In [49]:
# model.load_state_dict(torch.load(f'arch1_epoch25.pth'))

<All keys matched successfully>

In [21]:
preds = []
model.eval()
with torch.no_grad():
    for X, _ in tqdm(test_loader):
        features, logits = model(X.unsqueeze(1).to(torch.float32).cuda())
        classes = torch.argmax(logits, dim=1).cpu().numpy()
        preds.extend(classes)

100%|█████████████████████████████████████████████████████████████████████| 380/380 [00:38<00:00,  9.75it/s]


In [22]:
with open(pred_path, 'w') as f_pred:
    for idx, pred in enumerate(preds):
        name = test_helper.namelist[idx]
        cls = train_helper.vocabulary.class_by_index(pred)
        print(name, cls, file=f_pred)

In [23]:
!python -m course_ocr_t2.evaluate
# Accuracy = 0.7978

/bin/bash: /home/ichuviliaeva/miniconda3/envs/ocr_course/lib/python3.8/site-packages/cv2/../../../../lib/libtinfo.so.6: no version information available (required by /bin/bash)
pred_path =  /home/ichuviliaeva/ocr_course/course_ocr/task2/pred.txt
Accuracy = 0.8835


In [ ]:
# get 0.8755 on 20 epochs